## Validate GPU Installation (NVIDIA ONLY)

In [ ]:
!conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:54:10_Pacific_Daylight_Time_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.relgpu_drvr455TC455_06.29190527_0


## If conda install gpu doesnt work, use the following steps:
1. Go to this [link](https://developer.nvidia.com/rdp/cudnn-archive) and install cuDNN 8.1.1.
2. Extract the files.
3. Copy `bin\cudnn64_8.dll` → `C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.2\bin`
4. Copy `lib\x64\cudnn.lib` → `C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.2\lib\x64`
5. Copy `include\cudnn.h` → `C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.2\include`
6. Open **Windows Environment Variables**, and **Add these paths**:
```bash
C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.2\bin
C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.2\libnvvm\bin
C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.2\include
```
7. Verify
```bash
where cudnn64_8.dll
```

In [3]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Some more steps:
1. Move the nvvm folder into /models/research in the Tensorflow Object Detection API directory.

## Run fine-tuned model (ssd mobilenet v2 fpnlite 320x320 coco17)

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
from object_detection.utils import label_map_util

# Load the label map
label_map_path = "./mtsd_data/mtsd_label_map.pbtxt"
category_index = label_map_util.create_category_index_from_labelmap(label_map_path, use_display_name=True)

# Load the saved model
detect_fn = tf.saved_model.load("./models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/exported_model/saved_model")

# Function to draw bounding boxes and labels inside the boxes
def draw_boxes_with_labels(image, boxes, classes, scores, category_index, min_score_thresh=0.4):
    """
    Draws bounding boxes and labels inside the boxes on the given image.
    
    Args:
        image: The image to draw on.
        boxes: Bounding box coordinates (normalized) from the model.
        classes: Detected class indices.
        scores: Detection scores.
        category_index: Mapping of class indices to class names.
        min_score_thresh: Minimum score threshold for displaying detections.
    
    Returns:
        Annotated image with bounding boxes and labels inside.
    """
    height, width, _ = image.shape
    for i in range(len(boxes)):
        if scores[i] >= min_score_thresh:
            # Scale the box to image dimensions
            ymin, xmin, ymax, xmax = boxes[i]
            xmin = int(xmin * width)
            xmax = int(xmax * width)
            ymin = int(ymin * height)
            ymax = int(ymax * height)
            
            # Draw the rectangle
            color = (0, 255, 0)  # Green color
            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, 2)

            # Get the label text
            class_id = int(classes[i])
            class_name = category_index[class_id]['name']
            label = f"{class_name}: {scores[i]:.2f}"

            # Calculate text size
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            font_thickness = 1
            text_size = cv2.getTextSize(label, font, font_scale, font_thickness)[0]

            # Text background coordinates (inside the bounding box)
            text_x = xmin + 2
            text_y = ymin + text_size[1] + 2
            text_bg_x = text_x + text_size[0]
            text_bg_y = text_y - text_size[1]

            # Draw text background rectangle (inside the box)
            cv2.rectangle(image, (xmin, ymin), (text_bg_x, text_bg_y), color, -1)

            # Draw the label text (inside the box)
            cv2.putText(
                image,
                label,
                (text_x, text_y),
                font,
                font_scale,
                (0, 255, 0),  # Black text
                font_thickness,
                lineType=cv2.LINE_AA,
            )

    return image

# Load an image
image_path = "./mtsd_data/mtsd_images/02_PNG_jpg.rf.70c5e13b9fadb831af5abb7e7bf33c2e.jpg"
image = cv2.imread(image_path)

# Convert the image to a tensor
input_tensor = tf.convert_to_tensor(image)
input_tensor = input_tensor[tf.newaxis, ...]

# Perform detection
detections = detect_fn(input_tensor)

# Extract detection details
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
boxes = detections['detection_boxes']
scores = detections['detection_scores']
classes = detections['detection_classes'].astype(np.int32)

# Draw detections on the image
annotated_image = draw_boxes_with_labels(
    image=image.copy(), 
    boxes=boxes, 
    classes=classes, 
    scores=scores, 
    category_index=category_index, 
    min_score_thresh=0.4
)

# Display the image with detections
cv2.imshow("Detections", annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()